In [1]:
from util_TP206_v2 import *
from pylab import *
from soundfile import read
import os

fichier = "AJS_short.flac"
D = int(input("Durée de l'extrait en s.. "))
n = int(input("Quantification en combien de bits ? 4, 8, 16 ect.. bits "))
x = int(input("Facteur de sous-échantillonnage 2, 4, 8 ect.. "))

data, Fe = read(fichier)
extrait = data[0:Fe*D]
Fe_ss = Fe // x
extrait_ss = extrait[::x]

import sounddevice as sd
print("Lecture de l'audio sous-échantillonné...")
sd.play(extrait_ss, Fe_ss)
sd.wait()

if extrait_ss.ndim == 1:
    gauche = extrait_ss
    droite = extrait_ss
else:
    gauche = extrait_ss[:, 0]
    droite = extrait_ss[:, 1]

max_g = np.max(np.abs(gauche))
max_d = np.max(np.abs(droite))

gauche_norm = gauche / max_g if max_g != 0 else gauche
Droite_norm = droite / max_d if max_d != 0 else droite

niv = 2**n
qg, _ = uniform_quantizer(gauche_norm, niv, -1, 1)
qd, _ = uniform_quantizer(Droite_norm, niv, -1, 1)

rapportG = snr(gauche_norm, qg)
rapportD = snr(Droite_norm, qd)
print(f"SNR Gauche : {rapportG:.2f} dB")
print(f"SNR Droite : {rapportD:.2f} dB")

fe_min = 2 * 20000
if Fe_ss < fe_min:
    print("\nATTENTION : Le théorème de Shannon plus respecté !")
    print(f"Fe = {Fe_ss} Hz < {fe_min} Hz => Risque d'aliasing ou de recouvrement spectral")
else:
    print("\nOK : Shannon respecté")

tg = arange(0, len(gauche_norm)) / Fe_ss
td = arange(0, len(Droite_norm)) / Fe_ss

figure(figsize=(20, 10))

subplot(2,2,1)
plot(tg, gauche_norm)
title("Canal gauche original")
xlabel("Temps (s)")
ylabel("Amplitude")
grid()

subplot(2,2,2)
plot(td, Droite_norm)
title("Canal droit original")
xlabel("Temps (s)")
ylabel("Amplitude")
grid()

subplot(2,2,3)
plot(tg, qg)
title(f"Canal gauche quantifié ({n} bits)")
xlabel("Temps (s)")
ylabel("Amplitude")
grid()

subplot(2,2,4)
plot(td, qd)
title(f"Canal droit quantifié ({n} bits)")
xlabel("Temps (s)")
ylabel("Amplitude")
grid()

figure(figsize=(20,10))

freq = arange(-Fe_ss/2, Fe_ss/2, Fe_ss/len(gauche_norm))
spec_g = abs(fftshift(fft(gauche_norm))) / len(gauche_norm)
spec_qg = abs(fftshift(fft(qg))) / len(qg)

subplot(2,2,1)
plot(freq, spec_g)
title("Spectre gauche original")
grid()
xlim([-Fe_ss/8, Fe_ss/8])

subplot(2,2,3)
plot(freq, spec_qg)
title("Spectre gauche quantifié")
grid()
xlim([-Fe_ss/8, Fe_ss/8])

spec_d = abs(fftshift(fft(Droite_norm))) / len(Droite_norm)
spec_qd = abs(fftshift(fft(qd))) / len(qd)

subplot(2,2,2)
plot(freq, spec_d)
title("Spectre droit original")
grid()
xlim([-Fe_ss/8, Fe_ss/8])

subplot(2,2,4)
plot(freq, spec_qd)
title("Spectre droit quantifié")
grid()
xlim([-Fe_ss/8, Fe_ss/8])

tailleHD = len(data[0:Fe*D]) * 24
sortieQ = (len(qg) + len(qd)) * n
RC = tailleHD / sortieQ
print(f"Taux de compression : {RC:.2f}x")

if rapportG >= 20 and rapportD >= 20 and Fe_ss >= fe_min:
    print("\nCOMPROMIS IDEAL TROUVÉ")
    print(f"SNR G/D > 20 dB\n Shannon respecté\n Compression : {RC:.2f}x")
else:
    print("\nCOMPROMIS INSUFFISANT")
    if rapportG < 20 or rapportD < 20:
        print("SNR trop faible")
    if Fe_ss < fe_min:
        print("Fréquence trop basse : aliasing possible ou recouvrement spectral")


ModuleNotFoundError: No module named 'util_TP206_v2'

# Bon compromis atteint quand le taux est élevé (>3) tout en conservant un SNR > 20 dB